# Основное задание
Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data/kernels  
1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)  
2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)  
3.Добавьте аугментацию данных к пункту 2  
  
Сравните качество всех 3 полученных подходов  
  
# Задание со звездочкой  
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?  

import libraries

In [14]:
import torch
from torch import nn
import torchvision.models as models

import torchvision as tv
import time

import os

In [15]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
dev

device(type='cuda')

helpers

load data

In [17]:
data_dir = '../dll_hw_4_modern_cnn/hymenoptera_data/'

In [18]:
BATCH_SIZE = 32

In [19]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
#         tv.transforms.RandomResizedCrop(224),
#         tv.transforms.RandomHorizontalFlip(),
#         tv.transforms.RandomVerticalFlip(),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


create model

In [20]:
# import torchvision.models as models
# resnet18 = models.resnet18()
# alexnet = models.alexnet()
# vgg16 = models.vgg16()
# squeezenet = models.squeezenet1_0()
# densenet = models.densenet161()
# inception = models.inception_v3()
# googlenet = models.googlenet()
# shufflenet = models.shufflenet_v2_x1_0()
# mobilenet = models.mobilenet_v2()
# resnext50_32x4d = models.resnext50_32x4d()
# wide_resnet50_2 = models.wide_resnet50_2()
# mnasnet = models.mnasnet1_0()

In [21]:
# import torchvision.models as models
# resnet18 = models.resnet18(pretrained=True)
# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)

# 1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)

# ResNet 18

In [22]:
model = models.resnet18()

train model

In [23]:
model = model.to(dev)

In [24]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [25]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [26]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 3.697. Train acc: 0.000. Train Loss: 226.690
Step. time since epoch: 3.768. Train acc: 0.438. Train Loss: 190.569
Step. time since epoch: 3.838. Train acc: 0.562. Train Loss: 131.861
Step. time since epoch: 3.900. Train acc: 0.625. Train Loss: 85.485
Step. time since epoch: 3.965. Train acc: 0.625. Train Loss: 110.623
Step. time since epoch: 4.022. Train acc: 0.531. Train Loss: 103.986
Step. time since epoch: 4.078. Train acc: 0.562. Train Loss: 61.181
Step. time since epoch: 4.119. Train acc: 0.650. Train Loss: 24.302
epoch 1, loss 3.8307, train acc 0.492, test acc 0.340, time 7.2 sec
Step. time since epoch: 1.903. Train acc: 0.312. Train Loss: 649.054
Step. time since epoch: 2.004. Train acc: 0.531. Train Loss: 193.739
Step. time since epoch: 2.122. Train acc: 0.625. Train Loss: 146.253
Step. time since epoch: 2.240. Train acc: 0.469. Train Loss: 120.859
Step. time since epoch: 2.302. Train acc: 0.375. Train Loss: 51.031
Step. time since epoch: 2.360. Train ac

In [27]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16

In [28]:
model = models.vgg16()

train model

In [29]:
model = model.to(dev)

In [30]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [31]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [32]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 3.229. Train acc: 0.000. Train Loss: 222.352
Step. time since epoch: 3.440. Train acc: 0.469. Train Loss: 6032.982
Step. time since epoch: 3.820. Train acc: 0.656. Train Loss: 185.796
Step. time since epoch: 4.028. Train acc: 0.531. Train Loss: 250.057
Step. time since epoch: 4.237. Train acc: 0.438. Train Loss: 206.558
Step. time since epoch: 4.446. Train acc: 0.594. Train Loss: 206.309
Step. time since epoch: 4.654. Train acc: 0.406. Train Loss: 193.135
Step. time since epoch: 4.817. Train acc: 0.400. Train Loss: 99.354
epoch 1, loss 30.3137, train acc 0.439, test acc 0.542, time 7.7 sec
Step. time since epoch: 2.774. Train acc: 0.469. Train Loss: 150.171
Step. time since epoch: 2.984. Train acc: 0.500. Train Loss: 118.046
Step. time since epoch: 3.293. Train acc: 0.594. Train Loss: 128.473
Step. time since epoch: 3.603. Train acc: 0.469. Train Loss: 113.479
Step. time since epoch: 3.810. Train acc: 0.531. Train Loss: 100.342
Step. time since epoch: 4.019. Tra

# 2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)

# ResNet 18 pretrained

In [33]:
model = models.resnet18(pretrained=True)

train model

In [34]:
model = model.to(dev)

In [35]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [36]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [37]:
model.fc = nn.Linear(in_features=512, out_features=2).to(dev)

In [38]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [39]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [40]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [41]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 1.878. Train acc: 0.469. Train Loss: 26.402
Step. time since epoch: 1.979. Train acc: 0.656. Train Loss: 20.703
Step. time since epoch: 2.119. Train acc: 0.500. Train Loss: 27.007
Step. time since epoch: 2.192. Train acc: 0.562. Train Loss: 22.872
Step. time since epoch: 2.223. Train acc: 0.531. Train Loss: 22.296
Step. time since epoch: 2.254. Train acc: 0.688. Train Loss: 19.943
Step. time since epoch: 2.285. Train acc: 0.594. Train Loss: 21.159
Step. time since epoch: 2.305. Train acc: 0.600. Train Loss: 12.021
epoch 1, loss 0.7066, train acc 0.574, test acc 0.693, time 5.2 sec
Step. time since epoch: 2.026. Train acc: 0.719. Train Loss: 16.292
Step. time since epoch: 2.144. Train acc: 0.688. Train Loss: 18.135
Step. time since epoch: 2.258. Train acc: 0.719. Train Loss: 17.830
Step. time since epoch: 2.351. Train acc: 0.844. Train Loss: 14.947
Step. time since epoch: 2.383. Train acc: 0.812. Train Loss: 15.306
Step. time since epoch: 2.412. Train acc: 0.812.

In [42]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16 pretrained

In [43]:
model = models.vgg16(pretrained=True)

train model

In [44]:
model = model.to(dev)

In [45]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [46]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [47]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=2).to(dev)

In [48]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [49]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [50]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [51]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.207. Train acc: 0.469. Train Loss: 25.264
Step. time since epoch: 2.290. Train acc: 0.750. Train Loss: 17.225
Step. time since epoch: 2.395. Train acc: 0.938. Train Loss: 9.162
Step. time since epoch: 2.533. Train acc: 0.844. Train Loss: 9.913
Step. time since epoch: 2.602. Train acc: 0.906. Train Loss: 7.609
Step. time since epoch: 2.672. Train acc: 0.906. Train Loss: 6.628
Step. time since epoch: 2.741. Train acc: 1.000. Train Loss: 4.093
Step. time since epoch: 2.794. Train acc: 1.000. Train Loss: 3.739
epoch 1, loss 0.3428, train acc 0.844, test acc 0.961, time 6.0 sec
Step. time since epoch: 2.073. Train acc: 0.969. Train Loss: 5.691
Step. time since epoch: 2.214. Train acc: 1.000. Train Loss: 2.559
Step. time since epoch: 2.336. Train acc: 1.000. Train Loss: 1.660
Step. time since epoch: 2.489. Train acc: 1.000. Train Loss: 2.503
Step. time since epoch: 2.563. Train acc: 0.938. Train Loss: 3.082
Step. time since epoch: 2.631. Train acc: 0.969. Train Loss

# 3.Добавьте аугментацию данных к пункту 2  
  
(Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох))
  


In [52]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.RandomVerticalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


# ResNet 18 pretrained + aug

In [53]:
model = models.resnet18(pretrained=True)

train model

In [54]:
model = model.to(dev)

In [55]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [56]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [57]:
model.fc = nn.Linear(in_features=512, out_features=2).to(dev)

In [58]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [59]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [60]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [61]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 1.970. Train acc: 0.344. Train Loss: 39.011
Step. time since epoch: 2.070. Train acc: 0.469. Train Loss: 25.538
Step. time since epoch: 2.196. Train acc: 0.500. Train Loss: 22.707
Step. time since epoch: 2.335. Train acc: 0.344. Train Loss: 28.591
Step. time since epoch: 2.367. Train acc: 0.531. Train Loss: 26.591
Step. time since epoch: 2.396. Train acc: 0.438. Train Loss: 27.967
Step. time since epoch: 2.433. Train acc: 0.375. Train Loss: 25.145
Step. time since epoch: 2.454. Train acc: 0.400. Train Loss: 15.143
epoch 1, loss 0.8635, train acc 0.426, test acc 0.693, time 5.4 sec
Step. time since epoch: 2.001. Train acc: 0.500. Train Loss: 22.566
Step. time since epoch: 2.097. Train acc: 0.625. Train Loss: 21.733
Step. time since epoch: 2.316. Train acc: 0.688. Train Loss: 17.709
Step. time since epoch: 2.442. Train acc: 0.500. Train Loss: 24.128
Step. time since epoch: 2.472. Train acc: 0.594. Train Loss: 23.672
Step. time since epoch: 2.501. Train acc: 0.625.

In [62]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16 pretrained + aug

In [63]:
model = models.vgg16(pretrained=True)

train model

In [64]:
model = model.to(dev)

In [65]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [66]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [67]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=2).to(dev)

In [68]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [69]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [70]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [71]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.264. Train acc: 0.562. Train Loss: 22.031
Step. time since epoch: 2.340. Train acc: 0.688. Train Loss: 17.183
Step. time since epoch: 2.414. Train acc: 0.844. Train Loss: 13.662
Step. time since epoch: 2.495. Train acc: 0.812. Train Loss: 13.146
Step. time since epoch: 2.566. Train acc: 0.938. Train Loss: 7.616
Step. time since epoch: 2.637. Train acc: 0.906. Train Loss: 10.749
Step. time since epoch: 2.707. Train acc: 0.906. Train Loss: 7.158
Step. time since epoch: 2.761. Train acc: 0.800. Train Loss: 6.876
epoch 1, loss 0.4034, train acc 0.807, test acc 0.954, time 5.6 sec
Step. time since epoch: 2.252. Train acc: 0.938. Train Loss: 6.054
Step. time since epoch: 2.448. Train acc: 0.906. Train Loss: 5.840
Step. time since epoch: 2.556. Train acc: 0.969. Train Loss: 5.347
Step. time since epoch: 2.678. Train acc: 0.938. Train Loss: 3.871
Step. time since epoch: 2.751. Train acc: 0.875. Train Loss: 8.713
Step. time since epoch: 2.819. Train acc: 0.969. Train L

# Сравните качество всех 3 полученных подходов  


Качество обучения на тестовой и валидационной выборках в случае finetuning оказалось наилучшим. При этом предполагаем, что с учетом аугментации данных модель становится стабильнее.  

  
# Задание со звездочкой  
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?  